In [12]:
import ffmpeg
import os
import shutil
import numpy
from matplotlib import pyplot
import re
from datetime import datetime


In [13]:
TARGET_CODEC = 'mp3'
TARGET_BITRATE = '192k'
TARGET_BITRATE_INT = 192000


In [14]:
target_dir = 'A/A_001-050'

read_dir = f'../archive/tunes/{target_dir}'
write_dir = f'../compressed/tunes/{target_dir}'

os.makedirs(write_dir, exist_ok=True)

file_name_set = os.listdir(read_dir)


In [29]:
def print_log(log_file_name, log_props):
    log_line = ','.join([str(each) for each in log_props])
    print(log_line)
    with open(f'./{log_file_name}.log', 'a') as f:
        f.write(f'{log_line}\n')


def get_stream_info(read_file_path):
    stream_probe = ffmpeg.probe(read_file_path)['streams'][0]
    bit_rate = int(stream_probe['bit_rate'])
    duration = float(stream_probe['duration'])
    return (bit_rate, duration)


def define_action_type(duration, bit_rate, write_file_path) -> str:
    """
        file already exists                           -> pass
        file already exists but it seems to be broken -> copy
        file already has desired bps                  -> copy
        _                                             -> compress
    """
    if os.path.isfile(write_file_path):
        target_stream_probe = ffmpeg.probe(write_file_path)['streams'][0]
        target_duration = float(target_stream_probe['duration'])
        if abs(target_duration - duration) > 1:
            return 'COPY'
        else:
            return 'PASS'
    elif bit_rate <= TARGET_BITRATE_INT:
        return 'COPY'
    else:
        return 'COMPRESS'


def migrate(action_type, read_file_path, write_file_path):
    match action_type:
        case 'PASS':
            pass
        case 'COPY':
            shutil.copy(read_file_path, write_file_path)
        case 'COMPRESS':
            (
                ffmpeg
                .input(read_file_path)
                .output(
                    write_file_path,
                    format=TARGET_CODEC,
                    audio_bitrate=TARGET_BITRATE,
                )
                .run(quiet=True)
            )
        case _:
            raise ValueError

In [31]:
log_file_name = re.sub(':', '', datetime.now().isoformat().split('.')[0])

file_name = file_name_set[0]

read_file_path = f'{read_dir}/{file_name}'
write_file_path = f'{write_dir}/{file_name}'

(bit_rate, duration) = get_stream_info(read_file_path)
action_type = define_action_type(duration, bit_rate, write_file_path)
migrate(action_type, read_file_path, write_file_path)

log_props = (action_type, read_file_path, write_file_path, bit_rate, duration)
print_log(log_file_name, log_props)


PASS,../archive/tunes/A/A_001-050/00-A035 03 花紅葉.mp3,../compressed/tunes/A/A_001-050/00-A035 03 花紅葉.mp3,192000,747.102041
